# Solving Linear Systems in Julia

## A Consistent Linear System (n x n)
Where $Ax=b$ has an exact solution.

An example with random $A$ and $b$.

In [14]:
A = rand(3,3)
display(A)
b = rand(3,1)

3×3 Array{Float64,2}:
 0.10053   0.671944   0.866587
 0.060641  0.0186862  0.706712
 0.64705   0.466143   0.531028

3×1 Array{Float64,2}:
 0.2752780146684881
 0.6641174800899752
 0.8583483061009995

### Solving for $x$ with the built in "backsolve", "\".

In [15]:
A\b

3×1 Array{Float64,2}:
  1.2598917468059476
 -0.881392717887787 
  0.8549257098604145

### Using the $QR$ decomposition
and a home-made "back-substitution" function.

Here, we start with
$$A = QR$$

So
$$Ax = b$$
$$QRx = b$$
$$Rx = Q^{-1}b$$

Now, $Q$ is orthogonal, so we have
$$Rx = Q'b$$

And this is now in a form where we can employ back-substitution.


In [16]:
function backsub(U,y)
    # U is an upper triangular matrix
    # y is a column vector
    cols = size(U,2)
    b = fill(0.0, cols)
    Uf = convert(Array{Float64}, U)
    yf = convert(Array{Float64}, y)
    for i in range(cols, stop=1, step=-1)
        if i==cols
            b[i]=yf[i]/Uf[i,i]
        else
            yf[i] = yf[i] - transpose(Uf[i,:])*b
            b[i] = yf[i]/Uf[i, i]
        end
    end
    return b
end

backsub (generic function with 1 method)

In [11]:
using LinearAlgebra
F = qr(A,Val(true))
backsub(F.R, F.Q'*b)

3-element Array{Float64,1}:
 0.03372673679526412
 0.8102686163658064 
 0.2283920722750645 

## A Consistent Linear System (n x m, n > m)

More rows than columns.  Notice here that the $R$ we get back from `qr` is not conformable with $Q$.  Notice also that the solution from `\` is pivoted.


In [19]:
A = rand(4,3)
b = rand(4,1)
A\b

4×3 Array{Float64,2}:
 0.537679  0.622385  0.190834
 0.126656  0.586602  0.85736 
 0.266321  0.96532   0.937639
 0.788049  0.556113  0.880527

3×1 Array{Float64,2}:
  0.2730019409236663 
  1.0585704908913989 
 -0.24914533499175356

In [20]:
F = qr(A,Val(true))
display(F.R)
display(F.Q)
backsub(F.R, F.Q'*b)

3×3 Array{Float64,2}:
 -1.55756  -0.741424  -1.29465 
  0.0       0.668864   0.215861
  0.0       0.0       -0.499844

4×4 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.122521   0.668057  -0.639309   -0.360517
 -0.550452  -0.420806   0.0704361  -0.717609
 -0.601994  -0.269131  -0.488234    0.571664
 -0.565327   0.551535   0.589873    0.16812 

3-element Array{Float64,1}:
 -0.24914533499175362
  0.2730019409236663 
  1.0585704908913989 

## An Underdetermined System (n x n, rows not independent)
Where the rows beyond row m contain redundant information, i.e. the lower rows are each linear combinations of the preceding rows.

- Row 3 is just row 1+ row 2.

Here again, $R$ and $Q$ as returned by `qr` are not conformable, and the solution from `\` is pivoted.

In [30]:
A = [1 1;1 3; 2 4]
b = [3; 5; 8]
A\b

2-element Array{Float64,1}:
 1.9999999999999978
 1.0000000000000007

In [31]:
F = qr(A,Val(true))
display(F.R)
display(F.Q)
backsub(F.R, F.Q'*b)

2×2 Array{Float64,2}:
 -5.09902  -2.35339 
  0.0       0.679366

3×3 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.196116   0.792594  -0.57735
 -0.588348  -0.566139  -0.57735
 -0.784465   0.226455   0.57735

2-element Array{Float64,1}:
 1.0000000000000007
 1.9999999999999978

## An Underdetermined System (n x m, n < m)
Here again, $R$ and $Q$ as returned by `qr` are not conformable, and the solution from `\` is pivoted.

In [32]:
A = rand(3,4)
b = rand(3,1)
A\b

4×1 Array{Float64,2}:
 -0.34433059844453556
 -2.2502454378627363 
  2.6404066702272697 
  1.8076062582653212 

In [33]:
F = qr(A,Val(true))
display(F.R)
display(F.Q)
backsub(F.R, F.Q'*b)

3×4 Array{Float64,2}:
 -1.23302  -0.960355  -1.03638     -0.52832  
  0.0       0.650728   5.30181e-5   0.210882 
  0.0       0.0        0.195068     0.0996671

3×3 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.568796   0.650828  -0.502886
 -0.589175  -0.749041  -0.303004
 -0.573886   0.12394    0.809502

BoundsError: BoundsError: attempt to access 3×1 Array{Float64,2} at index [4]

In [42]:
Q = [F.Q; 0 0 0]'

3×4 Adjoint{Float64,Array{Float64,2}}:
 -0.568796  -0.589175  -0.573886  0.0
  0.650828  -0.749041   0.12394   0.0
 -0.502886  -0.303004   0.809502  0.0

In [43]:
backsub(F.R, Q'*b)

BoundsError: BoundsError: attempt to access 3×4 Array{Float64,2} at index [4, 4]